In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.wheel import wheel
from utils.calcium import calcium


Autosaving every 180 seconds


In [2]:
#############################################
######### LOAD SUITE2P BINARIZED DATA #######
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
#data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c.data_dir = data_dir
c.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/


#
c.load_binarization()
traces = c.F_onphase_bin    # c.F_upphase_bin

print ("binarized data (# cells, # time points): ", traces.shape)




  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (840, 55740)
         self.Fneu (neuropile):  (840, 55740)
         self.iscell (Suite2p cell classifier output):  (1098, 2)
              of which number of good cells:  (840,)
         self.spks (deconnvoved spikes):  (840, 55740)
         self.stat (footprints structure):  (840,)
         mean std over all cells :  13.0
   todo: print binarization defaults...
binarized data (# cells, # time points):  (840, 55740)


In [4]:
#################################################
####### OPTIONAL: REMOVE DUPLICATE CELLS ########
#################################################
c.load_footprints()
c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
c.corr_max_percent_overlap = 0.25       # max normalized cell body allowed 
c.corr_threshold = 0.3                  # max correlation allowed for high overlap; 
                                        #     note correlations computed using filtered fluorescecne not binarized
c.corr_delete_method = 'lowest_snr'     # highest_connected: removes hub neurons,keeps more cells; 
                                        # lowest_snr - removes lower SNR cells, keep less neurons
c.recompute_deduplication = False        # recompute the dedplucaiton wif new paramters are saved
c.n_cores = 8                           #  Number of cores to use for multi processing

#
c.remove_duplicate_neurons()            

#       
traces_unique = traces[c.clean_cell_ids]
print ("All cells: ", c.F.shape, "  unique cells: ", traces_unique.shape)


 ... deduplicating cells... (can take ~5 for ~1,000cells) 


getting correlations and pvals: 100%|█████████████████████████████████████████████████████████████████████████| 705600/705600 [00:01<00:00, 401854.45it/s]
14535it [00:00, 121154.24it/s]                                                                                                                            

All cells:  (840, 55740)   unique cells:  (608, 55740)


In [6]:
###############################################################
### OPTIONAL: LOAD WHEEL DATA AND QUEISCENT OR RUN PERIODS ####
###############################################################
# 
w = wheel.Wheel()
w.root_dir = os.path.join(c.data_dir.replace('suite2p/','').replace('plane0/',''),    # this assumes wheel data from Steffens' matlab scripts
                          'TRD-2P')                                                   #   is saved in the root diretory of the animal
w.load_track()
w.compute_velocity()
print ("Exp time : ", w.track.velocity.times.shape[0]/w.imaging_sample_rate)

# 
w.max_velocity_quiescent = 0.001  # in metres per second
idx_quiescent = w.get_indexes_quiescent_periods()

#
w.min_velocity_running = 0.1  # in metres per second
idx_run = w.get_indexes_run_periods()



Exp time :  1858.0
   Queiscent periods (seconds):  1341.6
   Running periods (seconds):  195.93333333333334


In [7]:
#########################################################
####### RUN PCA ON ALL OR SUBSET OF TRACES ##############
#########################################################
#

# take only 200 cells; and first 10 minutes
n_cells = 200
traces_unique = traces_unique[:n_cells]
print ("Selected cells (# cells, time points): ", traces_unique.shape)

# take quiescent periods
traces_in = traces_unique[:,idx_quiescent]

#
n_sec = 600
times = np.arange(n_sec)
traces_in = traces_in[:,times]

#
pca, X_pca = c.compute_PCA(traces_in)

# plot  variance exaplined
var_exp = pca.explained_variance_ratio_

# 
# plt.plot(var_exp)
# plt.ylabel("variance explained")
# plt.xlabel("PC component")
# plt.show()


Selected cells (# cells, time points):  (200, 55740)
